<a href="https://colab.research.google.com/github/chanlenium/Python/blob/main/CIS%20eFAS%20Industry%20DB%20Scraping/IT_EFAS_INDU_SPECIAL_DB/IT_EFAS_INDU_SPECIAL_DB_v7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
##이 부분은 처음 한번만 실행함

%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.lLFbM6lXGC/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.HDCahUR9rs/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.QXvGXKclO2/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

In [53]:
## Install library package (최초 한번만 실행)

!pip install yfinance
!pip install finance-datareader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [54]:
# import library
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

import requests
import pandas as pd
import numpy as np
from datetime import date

import xml.etree.ElementTree as ET
from urllib.request import urlopen

import yfinance as yf
import FinanceDataReader as fdr

today = date.today()

##### 조회년월
queryYYYYMM = '202301'

##### 조회분기
queryYYYYQQ = '2022Q4'

##### ECOS KEY
keyECOS = 'MTSA07N58C5X8EB4LMJL'

##### KOSIS KEY
keyKOSIS = '7AA431187D1220S0'

##### eNaraIndex KEY
keyENARA = '7AA431187D1220S0'

In [55]:
## ECOS API CALL function
def ecosApiCall(url_tail):
  url = f'https://ecos.bok.or.kr/api/StatisticSearch/{keyECOS}/json/kr/1/1000/{url_tail}'
  r = requests.get(url)
  jo = r.json()
  df = pd.DataFrame(jo['StatisticSearch']['row'])
  df = df[['TIME', 'DATA_VALUE']]
  df.rename(columns = {'TIME' : 'STD_YM'}, inplace = True)
  df['DATA_VALUE'] = pd.to_numeric(df['DATA_VALUE'])
  df = df.set_index('STD_YM')
  return df


## KOSIS API CALL function
def kosisApiCall(url):
  r = requests.get(url)
  return r.json()

In [56]:
## 자동차 산업 (출처 : 차동차 산업협회)
# MOTOR_PROD_QUAN : 자동차산업동향-생산수량(대)
# MOTOR_DOME_QUAN : 자동차산업동향-내수판매량(대)
# MOTOR_EXPORT_QUAN : 자동차산업동향-수출수량(대)
# MOTOR_EXPORT_SALES : 자동차산업동향-수출금액(천 달러)

#Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.
#Since Colab does not make web browser by default, set the web browser 
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome("/usr/bin/chromedriver", options=options)
#browser = webdriver.Chrome('chromedriver', options=options)

# 자동차 산업협회 홈페이지 url로 이동
url = "https://www.kama.or.kr/MainController" 
browser.get(url)
 
browser.find_element(By.XPATH, '//*[@id="gnb"]/li[1]/a[1]').click()
time.sleep(5)

browser.find_element(By.XPATH, '//*[@id="user_id"]').send_keys("kcredit")
browser.find_element(By.XPATH, '//*[@id="user_pw"]').send_keys("kcredit2019!")
browser.find_element(By.XPATH, '//*[@id="content"]/form/div/div/a').send_keys(Keys.ENTER)
time.sleep(10)


# 자동차통계DB(STATISTICS) 클릭
browser.find_element(By.XPATH, '//*[@id="wrap"]/div[2]/div/ul[2]/li[1]/a').send_keys(Keys.ENTER)
time.sleep(5)

# 조회조건 설정 : 2020년 1월 부터 최근년도 12월까지
select = Select(browser.find_element(By.XPATH, '//*[@id="statsYear"]'))
select.select_by_visible_text('2010년')
select.select_by_value('2010')
select = Select(browser.find_element(By.XPATH, '//*[@id="statsMonthE"]'))
select.select_by_visible_text('12월')
select.select_by_value('12')
browser.find_element(By.XPATH, '//*[@id="checkBox"]/table/tbody/tr[2]/td/label').click()
browser.find_element(By.XPATH, '//*[@id="checkBox"]/div/a').send_keys(Keys.ENTER)


print(browser.window_handles)
browser.switch_to.window(browser.window_handles[1])
time.sleep(5)

# Printing the whole body text
# print(browser.find_element(By.XPATH, "/html/body").text)

motorDataFrame = pd.read_html(browser.page_source)[2]
motorDataFrame.rename(columns=motorDataFrame.iloc[1], inplace =True)
motorDataFrame = motorDataFrame.iloc[2:]
motorDataFrame

# 브라우저 닫기
browser.close()

motorDataFrame = motorDataFrame[~motorDataFrame['월별'].isnull()] # NaN제거

# 월의 자릿수에 따라 앞에 '0'을 붙임
def assignMonth(row): 
  if int(row) >= 10: 
    result = row 
  else: 
    result = '0' + row 
  return result

motorDataFrame['STD_YM'] = motorDataFrame['년도'].str[0:-1:1].copy() + motorDataFrame['월별'].str[0:-1:1].copy().apply(assignMonth) # str[start:stop:step])


motorDataFrame = motorDataFrame[['STD_YM', '생산', '내수', '수출(수량)', '수출(금액)']]
motorDataFrame.rename(columns = {'생산' : 'MOTOR_PROD_QUAN', '내수' : 'MOTOR_DOME_QUAN', '수출(수량)' : 'MOTOR_EXPORT_QUAN', '수출(금액)' : 'MOTOR_EXPORT_SALES'}, inplace = True)
motorDataFrame.set_index('STD_YM', inplace =True)
motorDataFrame

<ipython-input-56-3a01922ac70e>:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome("/usr/bin/chromedriver", options=options)


['CDwindow-A7A4AE64B7A74BBAFD6FD2B8DF76255F', 'CDwindow-A0430FE4ECC13E4B1BFC0957622096EE']


,MOTOR_PROD_QUAN,MOTOR_DOME_QUAN,MOTOR_EXPORT_QUAN,MOTOR_EXPORT_SALES
STD_YM,,,,
201001,312847,119448,178578,2087684
201002,279368,106522,172760,2045657
201003,382173,123697,236266,2760632
201004,381959,123210,248165,2913250
201005,351819,116253,236497,2897538
...,...,...,...,...
202308,0,0,0,0
202309,0,0,0,0
202310,0,0,0,0


In [57]:
### 조선 산업 (출처 : e나라지표-수주량, 건조량, 수주잔량   /   클락슨-신조선가지수, 선복량, 해체량)
# SHIP_ORDER_AMOUNT : 선박 수주량(CGT)
# SHIP_TONNAGE : 선박 건조량(CGT)
# SHIP_KAJI : (클락슨)신조선가지수
# SHIP_BACKLOG : 선박 수주잔량(CGT)
# SHIP_BOTTOMS : (클락슨)선박 선복량(DWT)
# SHIP_BREAKUP : (클락슨)선박 해체량(DWT)
# (Ex) http://www.index.go.kr/openApi/xml_stts.do?userId=chanlenium&idntfcId=7AA431187D1220S0&statsCode=115102&period=201001:202211'

url = f'http://www.index.go.kr/openApi/xml_stts.do?userId=chanlenium&idntfcId={keyENARA}&statsCode=115102&period=201001:{queryYYYYMM}'
response = urlopen(url).read()
xtree = ET.fromstring(response)

## '표' tag에서 '주기'값이 '월'인 tree를 찾아서 월별 데이터 시작 지점을 찾음
for neighbor in xtree.iter("표"):
  if neighbor.attrib['주기'] == '월':
    rootMonth = neighbor

## '항목' tag에서 '이름'값에 따라 항목(수주랑, 건조량, 수주잔량)별 데이터 시작 지점을 찾음
for child in rootMonth.iter("항목"):
  if child.attrib['이름'] == '수주량':
    sooju = child
  elif child.attrib['이름'] == '건조량':
    gunjo = child
  elif child.attrib['이름'] == '수주잔량':
    soojujan = child

## 수주량 데이터 적제
period = [x.attrib["주기"] for x in sooju]
soojuValue = [x.text for x in sooju]

## 건조량 데이터 적제
gunjoValue = [x.text for x in gunjo]

## 수주잔량 데이터 적제
soojujanValue = [x.text for x in soojujan]

shipDataFrame = pd.DataFrame()
shipDataFrame['STD_YM'] = period
shipDataFrame['SHIP_ORDER_AMOUNT'] = soojuValue  # 수주량
shipDataFrame['SHIP_TONNAGE'] = gunjoValue  # 건조량
shipDataFrame['SHIP_BACKLOG'] = soojujanValue # 수주잔량

## 클락슨 데이터 들어갈 자리 삽입
shipDataFrame.insert(3, 'SHIP_KAJI', '')
shipDataFrame.insert(5, 'SHIP_BOTTOMS', '')
shipDataFrame["SHIP_BREAKUP"]=""

shipDataFrame = shipDataFrame.set_index('STD_YM')
shipDataFrame

,SHIP_ORDER_AMOUNT,SHIP_TONNAGE,SHIP_KAJI,SHIP_BACKLOG,SHIP_BOTTOMS,SHIP_BREAKUP
STD_YM,,,,,,
201501,79,147,,3475,,
201502,100,101,,3374,,
201503,116,81,,3333,,
201504,61,126,,3362,,
201505,120,91,,3296,,
...,...,...,...,...,...,...
202208,77,59,,3605,,
202209,132,74,,3624,,
202210,143,85,,3631,,


In [58]:
## 철강 산업 (출처 : KOSIS철강통계조사-조강생산량(천 톤)   /   철강협회-수출량, 내수판매량   /   Fred(IMF)-철광석가격)
# STEEL_EXPORT_QUAN : (철강협회)철강 수출량(톤)
# STEEL_DOME_QUAN : (철강협회)철강 내수판매량(톤)
# STEEL_CRUDE_QUAN : 조강생산량(천 톤)
# STEEL_PRICE : (Fred)철광석 가격(Million dollar per Metric Ton)

url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=NjgwMjNjNTc0NTBlZWM5Y2JjMmQ0YWEyNTIzMjhhNmM=&itmId=16363AAA0+&objL1=15363AA1AA+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe=201001&endPrdDe={queryYYYYMM}&orgId=363&tblId=TX_36301_A000'
ironDataFrame = pd.DataFrame(kosisApiCall(url))

# 원하는 컬럼만 뽑아냄(PRD_DE : 기준년월, DT : 조강생산량(천 톤))
ironDataFrame = ironDataFrame[['PRD_DE', 'DT']] 
ironDataFrame['DT'] = pd.to_numeric(ironDataFrame['DT'])
ironDataFrame.rename(columns = {'PRD_DE':'STD_YM'}, inplace = True)
ironDataFrame.rename(columns = {'DT':'STEEL_CRUDE_QUAN'}, inplace = True)
ironDataFrame = ironDataFrame.set_index('STD_YM')

# 철강협회 데이터 들어갈 자리 삽입
ironDataFrame.insert(0, 'STEEL_EXPORT_QUAN', '')
ironDataFrame.insert(1, 'STEEL_DOME_QUAN', '')

## 철광석 가격(Iron ore) : fred(https://fred.stlouisfed.org/)에서 PIORECRUSDM 데이터 가져옴
dfSteelPrice = fdr.DataReader('FRED:PIORECRUSDM', '2010-01')
dfSteelPrice['STD_YM'] = dfSteelPrice.index.astype(str).str.slice(0,4) + dfSteelPrice.index.astype(str).str.slice(5,7)
dfSteelPrice.rename(columns = {'PIORECRUSDM':'STEEL_PRICE'}, inplace = True)
dfSteelPrice = dfSteelPrice.set_index('STD_YM')

ironDataFrame = pd.merge(left = ironDataFrame, right = dfSteelPrice, left_index = True, right_index = True, how = "outer")
ironDataFrame


,STEEL_EXPORT_QUAN,STEEL_DOME_QUAN,STEEL_CRUDE_QUAN,STEEL_PRICE
STD_YM,,,,
201001,,,4570.0,125.914286
201002,,,4024.0,127.615000
201003,,,5087.0,139.769565
201004,,,5126.0,172.468182
201005,,,5246.0,161.347619
...,...,...,...,...
202209,,,4614.0,100.840909
202210,,,5151.0,94.619048
202211,,,4811.0,94.568182


In [59]:
## 석유화학 산업 (출처 : 한국석유화학협회-3대유도품수출량, 3대유도품내수판매량)
# PETRO_EXPORT_QUAN : (석유화학협회)3대유도품수출량
# PETRO_DOME_QUAN : (석유화학협회)3대유도품내수판매량
petroDataFrame = pd.DataFrame()
petroDataFrame['STD_YM'] = ironDataFrame.index
petroDataFrame['PETRO_EXPORT_QUAN'] = ""
petroDataFrame['PETRO_DOME_QUAN'] = ""
petroDataFrame = petroDataFrame.set_index('STD_YM')

petroDataFrame

,PETRO_EXPORT_QUAN,PETRO_DOME_QUAN
STD_YM,,
201001,,
201002,,
201003,,
201004,,
201005,,
...,...,...
202209,,
202210,,
202211,,


In [60]:
## 반도체 산업 (출처 : 인포맥스-수입현황,수출현황 등   /   야후파이낸스-필라델피아반도체지수)
# SEMI_PHILADELPHIA : 필라델피아지수

# 필라델피아지수
semiDataFrame = yf.download('^SOX',start = '2010-01-01')
semiDataFrame = semiDataFrame[['Adj Close']]
semiDataFrame['STD_YM'] = semiDataFrame.index.astype(str).str.slice(0,4) + semiDataFrame.index.astype(str).str.slice(5,7)
semiDataFrame = semiDataFrame.groupby("STD_YM")['Adj Close'].mean() # 월평균
semiDataFrame = semiDataFrame.to_frame()
semiDataFrame.rename(columns = {'Adj Close':'SEMI_PHILADELPHIA'}, inplace = True)

semiDataFrame

[*********************100%***********************]  1 of 1 completed


<ipython-input-60-2ece8312d494>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  semiDataFrame['STD_YM'] = semiDataFrame.index.astype(str).str.slice(0,4) + semiDataFrame.index.astype(str).str.slice(5,7)


,SEMI_PHILADELPHIA
STD_YM,
201001,349.100549
201002,331.668990
201003,358.039368
201004,386.359719
201005,356.888315
...,...
202211,2621.322405
202212,2652.837147
202301,2777.444995


In [61]:
## 정유 산업 (출처 : ECOS-수입현황,수출현황,내수판매량   /   싱가포르정제마진,두바이크랙마진)
# OIL_IMPORT_QUAN : 정유 수출입동향-수입
# OIL_EXPORT_QUAN  : 정유 수출입동향-수출
# OIL_DOME_QUAN : 정유 수출입동향-내수판매량
# OIL_CRACK_MARGIN : 두바이 크랙마진

# (1) 정유 수출입동향-수입
# 8.4.11. 석유제품수급 : [901Y073][M]   /   수입[I46B][천배럴]
# (Ex) https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/901Y073/M/201001/202211/I46B
dfOilImportQuan = ecosApiCall(f"901Y073/M/201001/{queryYYYYMM}/I46B")
dfOilImportQuan.rename(columns = {'DATA_VALUE':'OIL_IMPORT_QUAN'}, inplace = True)

# (2) 정유 수출입동향-수출
# 8.4.11. 석유제품수급 : [901Y073][M]   /   수출[I46D][천배럴]
# (Ex) https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/901Y073/M/201001/202209/I46B
dfOilExportQuan = ecosApiCall(f"901Y073/M/201001/{queryYYYYMM}/I46D")
dfOilExportQuan.rename(columns = {'DATA_VALUE':'OIL_EXPORT_QUAN'}, inplace = True)

# (3) 정유 수출입동향-내수판매
# 8.4.11. 석유제품수급 : [901Y073][M]   /   국내소비[I46C][천배럴]
# (Ex) https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/901Y073/M/201001/202209/I46B
dfOilDomeQuan = ecosApiCall(f"901Y073/M/201001/{queryYYYYMM}/I46C")
dfOilDomeQuan.rename(columns = {'DATA_VALUE':'OIL_DOME_QUAN'}, inplace = True)

# compile the list of dataframes you want to merge
df_list = [dfOilImportQuan, dfOilExportQuan, dfOilDomeQuan]
# grab first dataframe
oilDataFrame = df_list[0]
# loop through all but first data frame
for to_merge in df_list[1:]:
    # result of merge replaces first or previously merged data frame w/ all previous fields
    oilDataFrame = pd.merge(left = oilDataFrame, right = to_merge, left_index = True, right_index = True, how = "outer")

## 기타 데이터 삽입
oilDataFrame['OIL_CRACK_MARGIN'] = ""

oilDataFrame

,OIL_IMPORT_QUAN,OIL_EXPORT_QUAN,OIL_DOME_QUAN,OIL_CRACK_MARGIN
STD_YM,,,,
201001,100370,23532,69318,
201002,89701,22709,62758,
201003,90927,25249,66693,
201004,94342,28540,65588,
201005,99923,27471,65202,
...,...,...,...,...
202209,111377,42045,73919,
202210,110676,40504,75230,
202211,114417,39772,76785,


In [ ]:
## 해운 산업 (클락슨-BDI,SCFI,VLCC,CCFI   /   통계청KOSIS-컨테이너수송현황,수출입화물수송)
# HAEWOON_BDI : 해운 운임지수-BDI(벌크선)
# HAEWOON_SCFI : 해운 운임지수-SCFI(컨테이너선)
# HAEWOON_WS : 해운 운임지수-VLCC-WS(유조선)
# HAEWOON_CCFI : 해운 운임지수-CCFI
# HAEWOON_CONT : 해운 운임지수-컨테이너수송현황 (TEU)
# HAEWOON_TRANS : 해운 운임지수-수출입 화물수송 (R/T)
# HAEWOON_DUBAI : 두바이 유가
# HAEWOON_DOME_TRANS : 해운 국내 화물수송량 (톤)


## (통계청KOSIS) 컨테이너 화물수송 현황
startYYYYMM = 201101
endYYYYMM = int(queryYYYYMM)
yyyyMM_list = list(range(startYYYYMM, endYYYYMM, 100))

def kosisContainerApi(yyyyMM):
  s_yyyyMM = str(yyyyMM)
  e_yyyyMM = str(yyyyMM + 11)
  url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=NjgwMjNjNTc0NTBlZWM5Y2JjMmQ0YWEyNTIzMjhhNmM=&itmId=13103114674T.0003+&objL1=13102114674A.01+13102114674A.02+13102114674A.03+13102114674A.04+13102114674A.05+13102114674A.06+13102114674A.07+13102114674A.08+13102114674A.09+13102114674A.10+13102114674A.11+13102114674A.12+13102114674A.13+13102114674A.14+13102114674A.15+13102114674A.16+13102114674A.17+13102114674A.18+13102114674A.19+13102114674A.20+13102114674A.21+13102114674A.22+13102114674A.23+13102114674A.24+13102114674A.25+13102114674A.26+13102114674A.27+13102114674A.28+13102114674A.29+13102114674A.30+&objL2=13102114674B.01+13102114674B.02+&objL3=13102114674C.01+13102114674C.02+13102114674C.03+13102114674C.04+13102114674C.05+&objL4=13102114674D.0006+&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={s_yyyyMM}&endPrdDe={e_yyyyMM}&orgId=146&tblId=DT_MLTM_1312'
  r = requests.get(url)
  jo = r.json()
  df = pd.DataFrame(jo)
  df = df[['PRD_DE', 'C1_NM', 'C2_NM', 'C3_NM', 'DT']]  # C1_NM:항구명, C2_NM:국적구분(국적선/외국선), C3_NM:구분(내항/외항입항/외항출항/입항환적/출항환적, DT:화물수송량(R/T)
  df['DT'] = df['DT'].astype(float) # df['DT'] = pd.to_numeric(df['DT'])
  df.rename(columns = {'PRD_DE':'STD_YM', 'DT':'HAEWOON_CONT'}, inplace = True)
  df = df.set_index('STD_YM')
  return df

dfCONT = pd.DataFrame()
for elem in yyyyMM_list:
  if(dfCONT.empty):
    dfCONT = kosisContainerApi(elem)
  else:
    dfCONT = pd.concat([dfCONT, kosisContainerApi(elem)])
dfCONT = dfCONT.groupby('STD_YM').sum()



## (통계청KOSIS) 수출입 화물수송 현황
# (수출입 따로 보는 경우) url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=NjgwMjNjNTc0NTBlZWM5Y2JjMmQ0YWEyNTIzMjhhNmM=&itmId=13103114675T.0001+&objL1=13102114675A.01+&objL2=13102114675B.01+&objL3=13102114675C.0002+13102114675C.0003+&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe=201101&endPrdDe={queryYYYYMM}&orgId=146&tblId=DT_MLTM_1316'
# (수출입 합쳐 보는 경우) 
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=NjgwMjNjNTc0NTBlZWM5Y2JjMmQ0YWEyNTIzMjhhNmM=&itmId=13103114675T.0001+&objL1=13102114675A.01+&objL2=13102114675B.01+&objL3=13102114675C.0001+&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe=201101&endPrdDe={queryYYYYMM}&orgId=146&tblId=DT_MLTM_1316'
dfTRANS = pd.DataFrame(kosisApiCall(url))
dfTRANS = dfTRANS[['PRD_DE', 'C3_NM', 'DT']]  # (수출입 따로 보는경우) C3_NM:수출/수입/전체,   (수출입 같이 보는경우)C_NM:전체
dfTRANS['DT'] = round(pd.to_numeric(dfTRANS['DT']) / 1000, 0) # 천톤단위로 변환
dfTRANS = dfTRANS[['PRD_DE', 'DT']]
dfTRANS.rename(columns = {'PRD_DE':'STD_YM', 'DT':'HAEWOON_TRANS'}, inplace = True)
dfTRANS = dfTRANS.set_index('STD_YM')
# (수출입 따로 보는 경우)
#dfImport = pd.DataFrame()
#dfExport = pd.DataFrame()
#dfImport = df.query('C3_NM == "수입"')
#dfExport = df.query('C3_NM == "수출"')



## 유가(Dubai) 추가 (ECOS)
# 9.1.6.3. 국제상품가격 : [902Y003][M]   /   Dubai(현물) : [4010102][U$/bbl]
# (Ex) https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/902Y003/M/201001/202208/4010102
dfDUBAI = ecosApiCall(f"902Y003/M/201001/{queryYYYYMM}/4010102")
dfDUBAI.rename(columns = {'DATA_VALUE':'HAEWOON_DUBAI'}, inplace = True)



## (통계청KOSIS) 국내화물수송량 : 화물수송실적 - 내항화물입항현황 - 연안화물선(입항) - 화물(단위: R/T)
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=NjgwMjNjNTc0NTBlZWM5Y2JjMmQ0YWEyNTIzMjhhNmM=&itmId=13103114676T.0003+&objL1=13102114676A.01+&objL2=13102114676B.0002+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe=201006&endPrdDe={queryYYYYMM}&orgId=146&tblId=DT_MLTM_1314'
dfDomeTrans =  pd.DataFrame(kosisApiCall(url))
dfDomeTrans = dfDomeTrans[['PRD_DE', 'ITM_NM', 'C1_NM', 'C2_NM', 'DT']]  # ITM_NM:화물, C1_NM:총계, C2_NM:연안화물선(입항)
dfDomeTrans['DT'] = round(pd.to_numeric(dfDomeTrans['DT']) / 1000, 0) # 천톤단위로 변환
dfDomeTrans = dfDomeTrans[['PRD_DE', 'DT']]
dfDomeTrans.rename(columns = {'PRD_DE':'STD_YM', 'DT':'HAEWOON_DOME_TRANS'}, inplace = True)
dfDomeTrans = dfDomeTrans.set_index('STD_YM')



## (TradLinx) 컨테이너 운임지표 : CCFI, SCFI
#Since Colab does not make web browser by default, set the web browser 
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Set Head-less
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome("/usr/bin/chromedriver", options=options)
#browser = webdriver.Chrome('chromedriver', options=options)

# Data reference : Go to TradLinx homepage
url = "https://www.tradlinx.com/freight-index" 
browser.get(url)
 
# Select Month because we need monthly data
browser.find_element(By.XPATH, '/html/body/application/div/container-freight/div/div[2]/div/div[1]/div[3]/drp-input/div/div/span[2]').click()
browser.find_element(By.XPATH, '/html/body/application/div/container-freight/div/div[2]/div/div[1]/div[3]/drp-input/div/div[2]/ul/li[2]').click() 

print(browser.find_element(By.XPATH, '/html/body/application/div/container-freight/div/div[2]/div/div[1]/div[3]/drp-input/div/div/span[2]').text)

browser.find_element(By.XPATH, '/html/body/application/div/container-freight/div/div[2]/div/div[1]/div[4]/div/i').click() # Pick calendar
browser.implicitly_wait(5)
time.sleep(5)

# Set query condition : Select start day as '2020.1.1'
# Click 'JAN'
while browser.find_element(By.CLASS_NAME, 'dtp-actual-month').text != 'JAN':
  browser.find_element(By.CLASS_NAME, "dtp-select-month-before").click()
# Click '2010'
while browser.find_element(By.CLASS_NAME, 'dtp-actual-year').text != '2010':
  browser.find_element(By.CLASS_NAME, "dtp-select-year-before").click()
# Click 'day 1'
browser.find_element(By.XPATH, "/html/body/div[2]/div/div[1]/div[3]/div[1]/table/tbody/tr[2]/td[6]/a").click()

# Make sure that the selected day is 2010. JAN. 1
print(browser.find_element(By.CLASS_NAME, 'dtp-actual-year').text)
print(browser.find_element(By.CLASS_NAME, 'dtp-actual-month').text)
print(browser.find_element(By.XPATH, "/html/body/div[2]/div/div[1]/div[3]/div[1]/table/tbody/tr[2]/td[6]/a").text)
print(browser.find_element(By.CLASS_NAME, 'dtp-btn-ok').text)

browser.find_element(By.CLASS_NAME, 'dtp-btn-ok').click()
time.sleep(5)
#print(browser.find_element(By.XPATH, "/html/body").text)
containerDataFrame = pd.read_html(browser.page_source)[1]

containerDataFrame = containerDataFrame.transpose()
containerDataFrame.columns = containerDataFrame.iloc[0]
containerDataFrame = containerDataFrame[1:]
containerDataFrame['STDYM'] = containerDataFrame.index.str.replace(" ", "")

containerDataFrame.columns.name = None  # Delete column name 'Date All'
containerDataFrame.reset_index(drop=True, inplace=True) # index reset
containerDataFrame.set_index('STDYM', inplace=True) # Set index as 'STDYM'

dfSCFI = containerDataFrame[['SCFI']]
dfSCFI.rename(columns = {'SCFI':'HAEWOON_SCFI'}, inplace = True)
dfCCFI = containerDataFrame[['CCFI']]
dfCCFI.rename(columns = {'CCFI':'HAEWOON_CCFI'}, inplace = True)

# compile the list of dataframes you want to merge
df_list = [dfSCFI, dfCCFI, dfCONT, dfTRANS, dfDUBAI, dfDomeTrans]
# grab first dataframe
haewoonDataFrame = df_list[0]
# loop through all but first data frame
for to_merge in df_list[1:]:
    # result of merge replaces first or previously merged data frame w/ all previous fields
    haewoonDataFrame = pd.merge(left = haewoonDataFrame, right = to_merge, left_index = True, right_index = True, how = "outer")


# 기타 데이터 삽입
haewoonDataFrame.insert(0, 'HAEWOON_BDI', '')
haewoonDataFrame.insert(2, 'HAEWOON_WS', '')

haewoonDataFrame

<ipython-input-62-e840ea25c121>:83: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome("/usr/bin/chromedriver", options=options)


In [ ]:
## 건설 산업 (ECOS-건설국내수주량, 건설국내기성액, 건설해외수주량, 시도별건축착공현황(KOSIS), 미분양주택현황(서울,경기), 미분양주택현황(지방), 전국종합주택매매가격지수)
# CONST_DOME_ORDER_QUAN : 건설국내수주량
# CONST_REAL_PRICE : 건설국내기성액
# CONST_OVERSEA_ORDER_QUAN : 건설해외수주량
# CONST_BREAK_STATE : 시도별건축착공현황
# CONST_UNSOLD_URBAN : 미분양주택현황(서울,경기)
# CONST_UNSOLD_RURAL : 미분양주택현황(지방)
# CONST_HOUSE_PRICE : 전국종합주택매매가격지수

## 건설국내수주액
# 8.4.1. 국내건설수주액 : [901Y020][M]   /   총수주액[I42A][백만 원]
# (Ex) https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/901Y020/M/201001/202209/I42A
dfConstDomeOrderQuan = ecosApiCall(f"901Y020/M/201001/{queryYYYYMM}/I42A")
dfConstDomeOrderQuan.rename(columns = {'DATA_VALUE':'CONST_DOME_ORDER_QUAN'}, inplace = True)

## 국내건설기성액
# 8.4.2. 건설기성액 : [901Y104][M]   /   총기성액[I48A][백만 원]    /   경상[I37A]
# (Ex) https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/901Y104/M/201001/202209/I48A/I37A
dfConstRealPrice = ecosApiCall(f"901Y104/M/201001/{queryYYYYMM}/I48A/I37A")
dfConstRealPrice.rename(columns = {'DATA_VALUE':'CONST_REAL_PRICE'}, inplace = True)

## 시도별건축착공현황 - KOSIS
# https://kosis.kr/openapi/statisticsData.do?method=getList&apiKey=NjgwMjNjNTc0NTBlZWM5Y2JjMmQ0YWEyNTIzMjhhNmM=&format=json&jsonVD=Y&userStatsId=chanlenium/116/DT_MLTM_2202/2/2/20230216134719&prdSe=M&newEstPrdCnt=300&prdInterval=1
url = f'https://kosis.kr/openapi/statisticsData.do?method=getList&apiKey=NjgwMjNjNTc0NTBlZWM5Y2JjMmQ0YWEyNTIzMjhhNmM=&format=json&jsonVD=Y&userStatsId=chanlenium/116/DT_MLTM_2202/2/2/20230216134719&prdSe=M&newEstPrdCnt=500&prdInterval=1'
dfConstBreakState =  pd.DataFrame(kosisApiCall(url))
dfConstBreakState = dfConstBreakState[['PRD_DE', 'DT']]  #  "UNIT_NM": "동㎡"
dfConstBreakState = dfConstBreakState.loc[dfConstBreakState['PRD_DE'].astype(int) >= 201001]
dfConstBreakState.rename(columns = {'PRD_DE':'STD_YM', 'DT':'CONST_BREAK_STATE'}, inplace = True)
dfConstBreakState = dfConstBreakState.set_index('STD_YM')

## 미분양주택현황(서울,경기) 
# 8.4.5 미분양주택현황 : [901Y074][M]   /   서울[I410B][호], 경기[I410I][호]
# https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/901Y074/M/201001/202209/I410B
# https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/901Y074/M/201001/202209/I410I
dfConstUnsoldUrban = ecosApiCall(f"901Y074/M/201001/{queryYYYYMM}/I410B") + ecosApiCall(f"901Y074/M/201001/{queryYYYYMM}/I410I")
dfConstUnsoldUrban.rename(columns = {'DATA_VALUE':'CONST_UNSOLD_URBAN'}, inplace = True)
 
## 미분양주택현황(지방) 
# 8.4.5 미분양주택현황 : [901Y074][M]   /   전국[I410A][호]
# https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/901Y074/M/201001/202209/I410A
dfConstUnsoldRural = ecosApiCall(f"901Y074/M/201001/{queryYYYYMM}/I410A")
dfConstUnsoldRural.rename(columns = {'DATA_VALUE':'CONST_UNSOLD_RURAL'}, inplace = True)
mergeRuralUrban = pd.merge(left = dfConstUnsoldRural, right = dfConstUnsoldUrban, left_index = True, right_index = True)
mergeRuralUrban['CONST_UNSOLD_RURAL'] = mergeRuralUrban['CONST_UNSOLD_RURAL'] - mergeRuralUrban['CONST_UNSOLD_URBAN']
dfConstUnsoldRural = mergeRuralUrban['CONST_UNSOLD_RURAL']


## 건설해외수주량(출처: KOSIS)
url = f'http://www.index.go.kr/openApi/xml_stts.do?userId=chanlenium&idntfcId={keyKOSIS}&statsCode=122103&period=201001:{queryYYYYMM}'
response = urlopen(url).read()
xtree = ET.fromstring(response)

# '표' tag에서 '주기'값이 '월'인 tree를 찾아서 월별 데이터 시작 지점을 찾음
for neighbor in xtree.iter("표"):
  if neighbor.attrib['주기'] == '월':
    for elem in neighbor.iter("분류1"):
       if elem.attrib['이름'] == '합계':
         rootMonth = elem

# 해외건설수주액 데이터 적제
period = [x.attrib["주기"] for x in rootMonth]
value = [x.text for x in rootMonth]

dfConstOversea = pd.DataFrame()
dfConstOversea['STD_YM'] = period
dfConstOversea['CONST_OVERSEA_ORDER_QUAN'] = value
dfConstOversea['STD_YM'] = dfConstOversea['STD_YM'].astype(int)
dfConstOversea = dfConstOversea.query("STD_YM >= 201001")
dfConstOversea['STD_YM'] = dfConstOversea['STD_YM'].astype(str)
dfConstOversea = dfConstOversea.set_index('STD_YM')


## 전국종합주택매매가격지수(출처: KOSIS 유형별 매매가격지수)
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey=NjgwMjNjNTc0NTBlZWM5Y2JjMmQ0YWEyNTIzMjhhNmM=&itmId=sales+&objL1=00+&objL2=a0+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe=201001&endPrdDe={queryYYYYMM}&orgId=408&tblId=DT_40803_N0001'
dfConstHousePrice = pd.DataFrame(kosisApiCall(url))
dfConstHousePrice = dfConstHousePrice[['PRD_DE', 'C2_NM', 'DT']]  # 원하는 컬럼만 뽑아냄(PRD_DE : 기준년월, DT : DATA_VALUE(100 = 202106))
dfConstHousePrice['DT'] = pd.to_numeric(dfConstHousePrice['DT'])
dfConstHousePrice.rename(columns = {'PRD_DE':'STD_YM', 'DT':'CONST_HOUSE_PRICE'}, inplace = True)
dfConstHousePrice = dfConstHousePrice[['STD_YM', 'CONST_HOUSE_PRICE']]
dfConstHousePrice = dfConstHousePrice.set_index('STD_YM')

# compile the list of dataframes you want to merge
df_list = [dfConstDomeOrderQuan, dfConstRealPrice, dfConstOversea, dfConstBreakState, dfConstUnsoldUrban, dfConstUnsoldRural, dfConstHousePrice]
# grab first dataframe
constDataFrame = df_list[0]
# loop through all but first data frame
for to_merge in df_list[1:]:
    # result of merge replaces first or previously merged data frame w/ all previous fields
    constDataFrame = pd.merge(left = constDataFrame, right = to_merge, left_index = True, right_index = True, how = "outer")

constDataFrame

In [ ]:
## 모두 합치기
df_list = [motorDataFrame, shipDataFrame, ironDataFrame, petroDataFrame, semiDataFrame, oilDataFrame, haewoonDataFrame, constDataFrame]
# grab first dataframe
IT_EFAS_INDU_SPECIAL_DB = df_list[0]
# loop through all but first data frame
for to_merge in df_list[1:]:
    # result of merge replaces first or previously merged data frame w/ all previous fields
    IT_EFAS_INDU_SPECIAL_DB = pd.merge(left = IT_EFAS_INDU_SPECIAL_DB, right = to_merge, left_index = True, right_index = True, how = "outer")

today = date.today()
IT_EFAS_INDU_SPECIAL_DB.to_excel(f'{today}_IT_EFAS_INDU_SPECIAL_DB.xlsx')
IT_EFAS_INDU_SPECIAL_DB

In [ ]:
## 수기 입력한 데이터 합치기
## 순서 : 수기 입력한 엑셀파일을 업로드 -> 파일을 읽어들임(read_excel) -> 합치기 -> 엑셀파일 반출(to_excel)

# 수기 입력한 엑셀파일을 읽음(import)
BLANK_IT_EFAS_INDU_SPECIAL_DB = pd.read_excel('BLANK_IT_EFAS_INDU_SPECIAL_DB.xlsx')
BLANK_IT_EFAS_INDU_SPECIAL_DB = BLANK_IT_EFAS_INDU_SPECIAL_DB.set_index('STD_YM')

#BLANK_IT_EFAS_INDU_SPECIAL_DB와 IT_EFAS_INDU_SPECIAL_DB의 컬럼 길이를 맞추고 outerjoin 수행
IT_EFAS_INDU_SPECIAL_DB.index.name='STD_YM'
tempDf = pd.DataFrame(IT_EFAS_INDU_SPECIAL_DB.index)
tempDf['Temp'] = tempDf['STD_YM']
tempDf = tempDf.set_index('STD_YM')
tempDf.index = tempDf.index.astype('int64')

BLANK_IT_EFAS_INDU_SPECIAL_DB = pd.merge(left = BLANK_IT_EFAS_INDU_SPECIAL_DB, right = tempDf, left_index = True, right_index = True, how = "outer")
BLANK_IT_EFAS_INDU_SPECIAL_DB = BLANK_IT_EFAS_INDU_SPECIAL_DB.drop(['Temp'], axis = 1)
BLANK_IT_EFAS_INDU_SPECIAL_DB

# 수기 입력한 DB의 컬럼 이름을 리스트 형태로 변환: BLANK_IT_EFAS_INDU_SPECIAL_DB.columns.values.tolist()
#for columnName in BLANK_IT_EFAS_INDU_SPECIAL_DB.columns.values.tolist():
#    IT_EFAS_INDU_SPECIAL_DB[f'{columnName}'] = BLANK_IT_EFAS_INDU_SPECIAL_DB[f'{columnName}'].values

# 수기 입력한 Table의 값을 API Table로 복사
IT_EFAS_INDU_SPECIAL_DB[BLANK_IT_EFAS_INDU_SPECIAL_DB.columns.values.tolist()] = BLANK_IT_EFAS_INDU_SPECIAL_DB[BLANK_IT_EFAS_INDU_SPECIAL_DB.columns.values.tolist()].values

# 파일 반출
today = date.today()
IT_EFAS_INDU_SPECIAL_DB.to_excel(f'{today}_IT_EFAS_INDU_SPECIAL_DB.xlsx')
IT_EFAS_INDU_SPECIAL_DB